<a href="https://colab.research.google.com/github/alheliou/Bias_mitigation/blob/main/UPP26/TD4_pre_post.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TD 4: Mitigation des biais avec des méthodes de pré-processing et de post-processing



## Installation of the environnement

We highly recommend you to follow these steps, it will allow every student to work in an environment as similar as possible to the one used during testing.

### Colab Settings ---- for Colab Users ONLY
  The next cell of code are to execute only once per colab environment


#### Python env creation (Colab only)

        ```
        ! python -m pip install numpy fairlearn plotly nbformat ipykernel aif360["inFairness"] aif360['AdversarialDebiasing'] causal-learn BlackBoxAuditing cvxpy dice-ml lime shapkit
        ```
#### 2. Download MEPS dataset (for part2) it can take several minutes (Colab only)

        ```
        ! Rscript /usr/local/lib/python3.12/dist-packages/aif360/data/raw/meps/generate_data.R
        ! mv h181.csv /usr/local/lib/python3.12/dist-packages/aif360/data/raw/meps/
        ! mv h192.csv /usr/local/lib/python3.12/dist-packages/aif360/data/raw/meps/
        ```

### Local Settings ---- for installation on local computer ONLY

If you arleady have an env from TD2 or TD3, you can simply reuse it.


#### 1. Uv installation (local only, no need to redo if already done)


        https://docs.astral.sh/uv/getting-started/installation/


        `curl -LsSf https://astral.sh/uv/install.sh | sh`

        Python version 3.12 installation (highly recommended)
        `uv python install 3.12`

#### 2. R installation *NEW* (local only)

        In the command `Rscript` says 'command not found'

        `sudo apt install r-base-core`

#### 3. Python env creation (local only, no need to redo if already done)

        ```
        mkdir TD_bias_mitigation
        cd TD_bias_mitigation
        uv python pin 3.12
        uv init
        uv venv
        uv add numpy fairlearn plotly nbformat ipykernel aif360["inFairness"] aif360['AdversarialDebiasing'] causal-learn BlackBoxAuditing cvxpy dice-ml lime shapkit
        uv add pandas==2.2.2
        ```

#### 4. Download MEPS dataset, it can take several minutes *NEW* (local only)

        ```
        cd TD_bias_mitigation/.venv/lib/python3.12/site-packages/aif360/data/raw/meps/
        Rscript generate_data.R
        ```

In [ ]:
# To execute only in Colab
! python -m pip install numpy fairlearn plotly nbformat ipykernel aif360["inFairness"] aif360['AdversarialDebiasing'] causal-learn BlackBoxAuditing cvxpy dice-ml lime shapkit

In [ ]:
# To execute only in Colab
! Rscript /usr/local/lib/python3.12/dist-packages/aif360/data/raw/meps/generate_data.R
! mv h181.csv /usr/local/lib/python3.12/dist-packages/aif360/data/raw/meps/
! mv h192.csv /usr/local/lib/python3.12/dist-packages/aif360/data/raw/meps/

## 1.Manipulate the dataset

In [ ]:
# imports
import numpy as np
import pandas as pd
import plotly.express as px
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", append=True, category=UserWarning)
# Datasets
from aif360.datasets import MEPSDataset19
from aif360.explainers import MetricTextExplainer

# Fairness metrics
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from sklearn.metrics import accuracy_score, balanced_accuracy_score


MEPSDataset19_data = MEPSDataset19()
(dataset_orig_panel19_train, dataset_orig_panel19_val, dataset_orig_panel19_test) = (
    MEPSDataset19().split([0.5, 0.8], shuffle=True)
)

In [ ]:
len(dataset_orig_panel19_train.instance_weights), len(
    dataset_orig_panel19_val.instance_weights
), len(dataset_orig_panel19_test.instance_weights)

In [ ]:
instance_weights = MEPSDataset19_data.instance_weights
instance_weights

In [ ]:
f"Taille du dataset {len(instance_weights)}, poids total du dataset {instance_weights.sum()}."

Conversion en dataframe

In [ ]:
def get_df(MepsDataset):
    data = MepsDataset.convert_to_dataframe()
    # data_train est un tuple, avec le data_frame et un dictionnaire avec toutes les infos (poids, attributs sensibles etc)
    df = data[0]
    df["WEIGHT"] = data[1]["instance_weights"]
    return df


df = get_df(MEPSDataset19_data)

Nous réalisons maintenant l'opération inverse (qui sera indispensable pour le projet). Créer un objet de la classe StandardDataset de AIF360 à partir du dataframe.

Pour le projet cela vous permettre d'utiliser les méthode déjà implémentées dans AIF360 sur votre jeu de données.

Ici cela n'a aucun intéret car le dataframe vien d'un StandardDataset, nous vous fournissons le code. Mais cela vaut le coup de le lire attentivement et de poser des questions si besoin.




In [ ]:
import os
from aif360.datasets import StandardDataset
import pandas as pd

# Get categorical column from one hot encoding (specitic to MEPSdataset)
# Here we create a dictionnary that links each categorical column name
# to the list of corresponding one hot encoded columns
categorical_columns_dic = {}
for col in df.columns:
    col_split = col.split("=")
    if len(col_split) > 1:
        cat_col = col_split[0]
        if not (cat_col in categorical_columns_dic.keys()):
            categorical_columns_dic[cat_col] = []
        categorical_columns_dic[cat_col].append(col)
categorical_features = categorical_columns_dic.keys()

In [ ]:
# Now we recreate the categorical column value from the one hot encoded
print(df.shape)


def categorical_transform(df, onehotencoded, cat_col):
    if len(onehotencoded) > 1:
        return df[onehotencoded].apply(
            lambda x: onehotencoded[np.argmax(x)][len(cat_col) + 1 :], axis=1
        )
    else:
        return df[onehotencoded]


# Reverse the categorical one hot encoded
for cat_col, onehotencoded in categorical_columns_dic.items():
    df[cat_col] = categorical_transform(df, onehotencoded, cat_col)
    df.drop(columns=onehotencoded, inplace=True)

df.shape

In [ ]:
MyDataset = StandardDataset(
    df=df,
    label_name="UTILIZATION",
    favorable_classes=[1],
    protected_attribute_names=["RACE"],
    privileged_classes=[[1]],
    instance_weights_name="WEIGHT",
    categorical_features=categorical_features,
    features_to_keep=[],
    features_to_drop=[],
    na_values=["?", "Unknown/Invalid"],
    custom_preprocessing=None,
    metadata=None,
)

In [ ]:
# We check the dataset has the same metrics :D
# Attention étonnanement le positive label 'favorable_classes' est par défaut 1 (cela est un peu bizarre pour ce dataset)
print(
    BinaryLabelDatasetMetric(
        MEPSDataset19_data,
        unprivileged_groups=[{"RACE": 0}],
        privileged_groups=[{"RACE": 1}],
    ).disparate_impact(),
    BinaryLabelDatasetMetric(
        MEPSDataset19_data,
        unprivileged_groups=[{"RACE": 0}],
        privileged_groups=[{"RACE": 1}],
    ).base_rate(),
)
print(
    BinaryLabelDatasetMetric(
        MyDataset, unprivileged_groups=[{"RACE": 0}], privileged_groups=[{"RACE": 1}]
    ).disparate_impact(),
    BinaryLabelDatasetMetric(
        MyDataset, unprivileged_groups=[{"RACE": 0}], privileged_groups=[{"RACE": 1}]
    ).base_rate(),
)

In [ ]:
from aif360.sklearn.metrics import disparate_impact_ratio, base_rate

dir = disparate_impact_ratio(
    y_true=df.UTILIZATION, prot_attr=df.RACE, pos_label=1, sample_weight=df.WEIGHT
)
br = base_rate(y_true=df.UTILIZATION, pos_label=1, sample_weight=df.WEIGHT)
dir, br

## 2. Appliquer les méthodes de pré-processing disponibles dans AIF360

In [ ]:
sens_ind = 0
sens_attr = dataset_orig_panel19_train.protected_attribute_names[sens_ind]
unprivileged_groups = [
    {sens_attr: v}
    for v in dataset_orig_panel19_train.unprivileged_protected_attributes[sens_ind]
]
privileged_groups = [
    {sens_attr: v}
    for v in dataset_orig_panel19_train.privileged_protected_attributes[sens_ind]
]
sens_attr, unprivileged_groups, privileged_groups

### 2.1 Quesiton: Apprendre une regression logistique qui prédit l'UTILIZATION

Attention nous avons enlever le preprocessing sur le dataframe, il faut cette fois utiliser l'API d'AIF360
https://aif360.readthedocs.io/en/latest/modules/generated/aif360.datasets.StructuredDataset.html

pour retrouver les features (X), les labels (y) et les poids de chaque instance du dataset

In [ ]:
print("TODO")

### 2.2 Question: Calcul des métriques de fairness

Calculer les métriques du dataset de validation seul.

Calculer les métriques basées sur les prédictions et la vérité du dataset de validation.

En comparaison calculer les métriques basées sur des prédictions aléatoires et la vérité du dataset de validation.

In [ ]:
print("TODO Metrics on validation dataset")

In [ ]:
print("TODO Metrics based on predictions and truth on validation dataset")

In [ ]:
print("TODO Metrics based on random predictions and truth on validation dataset")

### 2.2 Repondération
#### 2.2.1. Question : Trouver dans l'API quels objets/fonctions sont à utiliser pour faire de repondération et les appliquer sur le dataset d'apprentissage

In [ ]:
print("TODO")

#### 2.2.2. Question: Apprendre une regression logistique sur les données pondérées et calculer les métriques de fairness sur l'échantillon de validation

Comme vu en cours le Reweighting ne modifie que la pondération du dataset, les features et label restent inchangés.

In [ ]:
print("TODO")

### 2.3. Disparate Impact Remover
#### 2.3.1. Question : Trouver dans l'API quels objets/fonctions sont à utiliser pour faire une approache de disparate impact remover et les appliquer.


In [ ]:
print("TODO")

#### 2.3.2. Question: Apprendre une regression logistique sur les données transformées en retirant l'attribut sensible et calculer les métriques de fairness sur l'échantillon de validation

In [ ]:
print("TODO")

### 2.4. Question: Apprentissage de représentation latente fair

Apprendre le pre-processing et evaluer son impact avec les métriques

In [ ]:
print("TODO")

## 3 Post processing

### 3.1 Question: Use the post-processing Reject Option Classification

In [ ]:
print("TODO")

#### 3.1.1 Reuse the first Logistic Regression learn to find the best threshold that maximises its balanced accuracy on the validation dataset

In [ ]:
print("TODO")

#### 3.1.2 Use the RejectOptionClassification  on the validation dataset with the logistic regression predictions. To improve the fairness metrics

In [ ]:
print("TODO")

#### 3.1.3 Do the same while starting from the Logistic Regression learned on the Reweighted dataset

In [ ]:
print("TODO")

#### 3.2 Use the Calibrated Equalised Odds  on the validation dataset with the logistic regression predictions. To improve the fairness metrics

In [ ]:
print("TODO")